In [1]:
import sys
import os
import numpy as np
import pandas as pd
import json
# mcfly
from mcfly import modelgen, find_architecture, storage

Using TensorFlow backend.


In [2]:
data_path = '/media/sf_VBox_Shared/timeseries/actitiracker/WISDM_at_v2.0/'
preprocessed_path = os.path.join(data_path, 'preprocessed')
result_path = os.path.join(data_path, 'models_test')

In [3]:
X_train = np.load(os.path.join(preprocessed_path, 'X_train.npy'))
X_val = np.load(os.path.join(preprocessed_path, 'X_val.npy'))
X_test = np.load(os.path.join(preprocessed_path, 'X_test.npy'))
y_train = np.load(os.path.join(preprocessed_path, 'y_train.npy'))
y_val = np.load(os.path.join(preprocessed_path, 'y_val.npy'))
y_test = np.load(os.path.join(preprocessed_path, 'y_test.npy'))

In [4]:
with open(os.path.join(preprocessed_path, 'labels.json')) as f:
    labels = json.load(f)

## Generate models

In [10]:
num_classes = y_train.shape[1]

models = modelgen.generate_models(X_train.shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 2) #10)

In [11]:
#what is the fraction of classes in the validation set?
pd.Series(y_val.mean(axis=0), index=labels)

Walking      0.445402
LyingDown    0.055904
Standing     0.082027
Sitting      0.281609
Jogging      0.103971
Stairs       0.031087
dtype: float64

In [12]:
if not os.path.exists(result_path):
        os.makedirs(result_path)

In [ ]:
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train,
                                                                           X_val[:10], y_val[:10],
                                                                           models,nr_epochs=1, #5,
                                                                           subset_size=128, # 512,
                                                                           verbose=True,
                                                                           batch_size=32,
                                                                           outputpath=result_path,
                                                                           early_stopping=True)

In [14]:
print('Details of the training process were stored in ',os.path.join(result_path, 'models.json'))

Details of the training process were stored in  /media/sf_VBox_Shared/timeseries/actitiracker/WISDM_at_v2.0/models_test/models.json


In [15]:
best_model_index = np.argmax(val_accuracies)
best_model, best_params, best_model_types = models[best_model_index]
print('Model type and parameters of the best model:')
print(best_model_types)
print(best_params)


Model type and parameters of the best model:
DeepConvLSTM
{'regularization_rate': 0.00574537358824132, 'lstm_dims': [48, 52, 42, 38, 56], 'filters': [100, 87, 31, 82, 70], 'learning_rate': 0.0011995620624020058}


In [17]:
nr_epochs = 3
datasize = 128 # X_train.shape[0]
history = best_model.fit(X_train[:datasize,:,:], y_train[:datasize,:],
              epochs=nr_epochs, validation_data=(X_val, y_val))

Train on 128 samples, validate on 3828 samples
Epoch 1/3
 96/128 [=====================>........] - ETA: 9s - loss: 3.5834 - acc: 0.4792 

KeyboardInterrupt: 

In [20]:
best_model.save(os.path.join(result_path, 'best_model.h5'))

In [22]:
## Test on Testset
score_test = best_model.evaluate(X_test, y_test, verbose=True)
print('Score of best model: ' + str(score_test))

1641/1641 [==============================] - 157s   
Score of best model: [3.3572144339827048, 0.57586837305229366]
